# Wind Turbine combined with Heat Pump and Water Tank

In this example the heat demand is supplied by a wind turbine in combination with a heat pump and a water tank that stores hot water with a standing loss.

In [1]:
import pandas as pd

import pypsa

In [2]:
network = pypsa.Network()
network.set_snapshots(pd.date_range("2016-01-01 00:00", "2016-01-01 03:00", freq="H"))

network.add("Bus", "0", carrier="AC")
network.add("Bus", "0 heat", carrier="heat")

network.add("Carrier", "wind")
network.add("Carrier", "heat")

network.add(
    "Generator",
    "wind turbine",
    bus="0",
    carrier="wind",
    p_nom_extendable=True,
    p_max_pu=[0.0, 0.2, 0.7, 0.4],
    capital_cost=500,
)

network.add("Load", "heat demand", bus="0 heat", p_set=20.0)

# NB: Heat pump has changing efficiency (properly the Coefficient of Performance, COP)
# due to changing ambient temperature
network.add(
    "Link",
    "heat pump",
    bus0="0",
    bus1="0 heat",
    efficiency=[2.5, 3.0, 3.2, 3.0],
    capital_cost=1000,
    p_nom_extendable=True,
)

network.add(
    "Store",
    "water tank",
    bus="0 heat",
    e_cyclic=True,
    e_nom_extendable=True,
    standing_loss=0.01,
)

C:\Users\dell\AppData\Local\Temp\ipykernel_11012\743167227.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  network.set_snapshots(pd.date_range("2016-01-01 00:00", "2016-01-01 03:00", freq="H"))


Index(['water tank'], dtype='object')

In [3]:
network.optimize()

Index(['0'], dtype='object', name='Bus')
Index(['heat pump'], dtype='object', name='Link')
Index(['0'], dtype='object', name='Bus')
Index(['heat pump'], dtype='object', name='Link')
C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\linopy\common.py:147: UserWarning: coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.
  warn(
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.05s
INFO:linopy.solvers:Log file at C:\Users\dell\AppData\Local\Temp\highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 19 primals, 40 duals
Objective: 2.35e+04
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-

('ok', 'optimal')

In [4]:
pd.DataFrame({attr: network.stores_t[attr]["water tank"] for attr in ["p", "e"]})

,p,e
snapshot,,
2016-01-01 00:00:00,20.000000,4.376710
2016-01-01 01:00:00,4.332943,-0.000000
2016-01-01 02:00:00,-13.423055,13.423055
2016-01-01 03:00:00,-11.334114,24.622939


In [5]:
pd.DataFrame({attr: network.links_t[attr]["heat pump"] for attr in ["p0", "p1"]})

,p0,p1
snapshot,,
2016-01-01 00:00:00,-0.000000,0.000000
2016-01-01 01:00:00,5.222352,-15.667057
2016-01-01 02:00:00,10.444705,-33.423055
2016-01-01 03:00:00,10.444705,-31.334114


In [6]:
network.stores.loc[["water tank"]].T

Store,water tank
bus,0 heat
type,
carrier,heat
e_nom,0.0
e_nom_mod,0.0
e_nom_extendable,True
e_nom_min,0.0
e_nom_max,inf
e_min_pu,0.0
e_max_pu,1.0


In [7]:
network.generators.loc[["wind turbine"]].T

Generator,wind turbine
bus,0
control,PQ
type,
p_nom,0.0
p_nom_mod,0.0
p_nom_extendable,True
p_nom_min,0.0
p_nom_max,inf
p_min_pu,0.0
p_max_pu,1.0
